# Predict Pipeline

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
import torchvision.transforms as T
from PIL import Image
import numpy as np
import json
import csv
from pathlib import Path
from tqdm import tqdm
import timm
import pickle
import warnings

warnings.filterwarnings('ignore')

In [ ]:
class TestPredictor:
    def __init__(self, model_dir, device='cuda'):
        self.model_dir = Path(model_dir)
        self.device = torch.device(device if torch.cuda.is_available() else 'cpu')

        self.class_names = [
            "nose-right", "nose-left", "ear-right",
            "ear-left", "vc-open", "vc-closed", "throat"
        ]
        self.num_classes = len(self.class_names)

        print(f"Using device: {self.device}")

    def load_ensemble_models(self):
        print("Loading ensemble models...")

        with open(self.model_dir / 'ensemble_info.pkl', 'rb') as f:
            ensemble_info = pickle.load(f)

        models = []
        model_names = ensemble_info['models']
        weights = ensemble_info['weights']

        for i, model_name in enumerate(model_names):
            print(f"Loading model {i+1}/{len(model_names)}: {model_name}")

            base_name = "convnext_base.fb_in22k_ft_in1k"

            model = timm.create_model(base_name, pretrained=False, num_classes=self.num_classes)
            state_dict = torch.load(self.model_dir / f"ensemble_model_{i}.pt", map_location=self.device)
            model.load_state_dict(state_dict)
            model.to(self.device)
            model.eval()

            models.append({'model': model, 'weight': weights[i], 'name': model_name})

        print(f"Loaded {len(models)} models successfully.")
        return models

    def load_test_data(self, csv_path, img_dir):
        test_files = []
        with open(csv_path, 'r') as f:
            reader = csv.reader(f)
            for row in reader:
                if row:
                    img_name = row[0].strip()
                    img_path = Path(img_dir) / img_name
                    if img_path.exists():
                        test_files.append(str(img_path))
                    else:
                        print(f"Warning: Image not found: {img_path}")

        print(f"Loaded {len(test_files)} test images.")
        return test_files

    def get_tta_transforms(self, img_size=224, n_aug=5):
        class ResizeOrPad:
            def __init__(self, min_size):
                self.min_size = min_size

            def __call__(self, img):
                w, h = img.size
                if w < self.min_size or h < self.min_size:
                    scale = self.min_size / min(w, h)
                    new_w = int(w * scale)
                    new_h = int(h * scale)
                    return T.functional.resize(img, (new_h, new_w))
                return img

        transforms = [
            T.Compose([
                ResizeOrPad(img_size),
                T.Resize((img_size, img_size)),
                T.ToTensor(),
                T.Normalize(mean=[0.485, 0.456, 0.406],
                            std=[0.229, 0.224, 0.225])
            ])
        ]

        import random
        for i in range(n_aug - 1):
            resize_delta = random.choice([-20, -10, 0, 10, 20])
            target_size = max(img_size, img_size + resize_delta)

            transforms.append(
                T.Compose([
                    ResizeOrPad(target_size + 20),
                    T.Resize((target_size + 10, target_size + 10)),
                    T.CenterCrop(img_size) if i % 2 == 0 else T.RandomCrop(img_size),
                    T.RandomApply([
                        T.ColorJitter(
                            brightness=random.uniform(0.1, 0.2),
                            contrast=random.uniform(0.1, 0.2),
                            saturation=random.uniform(0.05, 0.15),
                            hue=random.uniform(0.02, 0.05)
                        )
                    ], p=0.8),
                    T.RandomChoice([
                        T.GaussianBlur(3, sigma=(0.1, 0.5)),
                        nn.Identity()
                    ]),
                    T.ToTensor(),
                    T.Normalize(mean=[0.485, 0.456, 0.406],
                                std=[0.229, 0.224, 0.225])
                ])
            )
        return transforms

    def predict_single_image_tta(self, model, img_path, n_aug=5):
        image = Image.open(img_path).convert('RGB')
        tta_transforms = self.get_tta_transforms(n_aug=n_aug)

        aug_probs = []
        with torch.no_grad():
            for transform in tta_transforms:
                img_tensor = transform(image).unsqueeze(0).to(self.device)
                output = model(img_tensor)
                probs = F.softmax(output, dim=1)
                aug_probs.append(probs)

        weights = torch.tensor([1.5] + [1.0] * (n_aug - 1)).to(self.device)
        weights = weights / weights.sum()

        final_probs = torch.zeros_like(aug_probs[0])
        for i, probs in enumerate(aug_probs):
            final_probs += probs * weights[i]

        return final_probs.cpu().numpy().squeeze()

    def ensemble_predict_batch(self, models, test_files, use_tta=True, batch_size=32):
        predictions = {}
        for img_path in tqdm(test_files, desc="Predicting"):
            img_name = Path(img_path).name
            all_model_probs = []
            model_weights = []

            for model_info in models:
                model = model_info['model']
                weight = model_info['weight']

                if use_tta:
                    probs = self.predict_single_image_tta(model, img_path, n_aug=3)
                else:
                    transform = T.Compose([
                        T.Resize((224, 224)),
                        T.ToTensor(),
                        T.Normalize(mean=[0.485, 0.456, 0.406],
                                    std=[0.229, 0.224, 0.225])
                    ])
                    image = Image.open(img_path).convert('RGB')
                    img_tensor = transform(image).unsqueeze(0).to(self.device)
                    with torch.no_grad():
                        output = model(img_tensor)
                        probs = F.softmax(output, dim=1).cpu().numpy().squeeze()

                all_model_probs.append(probs)
                model_weights.append(weight ** 2)

            model_weights = np.array(model_weights)
            model_weights /= model_weights.sum()

            final_probs = np.zeros_like(all_model_probs[0])
            for i, probs in enumerate(all_model_probs):
                final_probs += probs * model_weights[i]

            pred_class = np.argmax(final_probs)
            predictions[img_name] = int(pred_class)

        return predictions

    def predict_and_save(self, csv_path, img_dir, output_path, use_tta=True):
        models = self.load_ensemble_models()
        test_files = self.load_test_data(csv_path, img_dir)

        print("\nStarting prediction...")
        predictions = self.ensemble_predict_batch(models, test_files, use_tta=use_tta)

        print(f"\nSaving results to {output_path}")
        with open(output_path, 'w') as f:
            json.dump(predictions, f, indent=2)

        print(f"Saved {len(predictions)} predictions")

        print("\nSample predictions:")
        for i, (img_name, pred) in enumerate(list(predictions.items())[:5]):
            print(f"  {img_name}: {pred} ({self.class_names[pred]})")

        return predictions

In [ ]:
model_dir = "/kaggle/input/convnextbase-ensemble-metalearner"
csv_path = "//kaggle/input/ent-private-test/ENTRep_Track1_Private_Data/cls.csv"
img_dir = "/kaggle/input/ent-private-test/ENTRep_Private_Dataset_Update/imgs"
output_path = "/kaggle/working/predictions.json"

predictor = TestPredictor(model_dir)
predictions = predictor.predict_and_save(
    csv_path=csv_path,
    img_dir=img_dir,
    output_path=output_path,
    use_tta=True
)

print(f"Results saved to: {output_path}")

# Predict Pipeline cải tiến

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
import torchvision.transforms as T
from PIL import Image
import numpy as np
import json
import csv
from pathlib import Path
from tqdm import tqdm
import timm
import pickle
import warnings
import random

warnings.filterwarnings('ignore')

try:
    from peft import LoraConfig, get_peft_model, PeftModel
    PEFT_AVAILABLE = True
    print("PEFT library is available.")
except ImportError:
    PEFT_AVAILABLE = False
    print("Warning: PEFT library not found. Model loading might fail if PEFT was used during training.")

In [ ]:
class ResizeOrPad:
    def __init__(self, min_size):
        self.min_size = min_size

    def __call__(self, img):
        w, h = img.size
        if w < self.min_size or h < self.min_size:
            scale = self.min_size / min(w, h)
            new_w = int(w * scale)
            new_h = int(h * scale)
            return T.functional.resize(img, (new_h, new_w))
        return img

class AnatomicalGroupAuxiliaryModel(nn.Module):
    def __init__(self, backbone_name, num_classes=7, num_groups=4, drop_rate=0.4, drop_path_rate=0.3,
                 use_peft=True, lora_rank=16, lora_alpha=32, lora_dropout=0.1):
        super().__init__()
        self.use_peft = use_peft and PEFT_AVAILABLE

        self.backbone = timm.create_model(
            backbone_name, pretrained=False, num_classes=0,
            drop_rate=drop_rate, drop_path_rate=drop_path_rate
        )
        feature_dim = self.backbone.num_features

        self.primary_head = nn.Sequential(nn.Dropout(drop_rate), nn.Linear(feature_dim, num_classes))
        self.auxiliary_head = nn.Sequential(nn.Dropout(drop_rate), nn.Linear(feature_dim, num_groups))

        if self.use_peft:
            self._apply_peft(lora_rank, lora_alpha, lora_dropout)

    def _apply_peft(self, rank, alpha, dropout):
        if not PEFT_AVAILABLE: return
        target_modules = [name for name, module in self.backbone.named_modules() if isinstance(module, nn.Linear)]
        peft_config = LoraConfig(
            r=rank, lora_alpha=alpha, lora_dropout=dropout,
            target_modules=target_modules, bias="none"
        )
        self.backbone = get_peft_model(self.backbone, peft_config)

    def forward(self, x):
        features = self.backbone(x)
        primary_logits = self.primary_head(features)
        auxiliary_logits = self.auxiliary_head(features)
        return primary_logits, auxiliary_logits

In [ ]:
class TestPredictor:
    def __init__(self, model_dir, device='cuda'):
        self.model_dir = Path(model_dir)
        self.device = torch.device(device if torch.cuda.is_available() else 'cpu')

        self.class_names = [
            "nose-right", "nose-left", "ear-right",
            "ear-left", "vc-open", "vc-closed", "throat"
        ]
        self.num_classes = len(self.class_names)
        print(f"Using device: {self.device}")

    def load_ensemble_models(self):
        print("Loading PEFT-trained ensemble models...")

        try:
            with open(self.model_dir / 'ensemble_info.pkl', 'rb') as f:
                ensemble_info = pickle.load(f)
        except FileNotFoundError:
            print(f"Error: 'ensemble_info.pkl' not found in {self.model_dir}")
            return []

        models = []
        model_names = ensemble_info.get('models', [])
        weights = ensemble_info.get('weights', [])
        peft_config = ensemble_info.get('peft_config', {'use_peft': True, 'rank': 16, 'alpha': 32})

        for i, model_name in enumerate(model_names):
            print(f"Loading model {i+1}/{len(model_names)}: {model_name}")

            base_name = model_name
            if base_name.endswith('_full'):
                base_name = base_name[:-5]
            elif '_fold' in base_name:
                base_name = base_name.rsplit('_', 1)[0]

            backbone_name = base_name

            model = AnatomicalGroupAuxiliaryModel(
                backbone_name=backbone_name,
                num_classes=self.num_classes,
                num_groups=4,
                use_peft=peft_config.get('use_peft', True),
                lora_rank=peft_config.get('rank', 16),
                lora_alpha=peft_config.get('alpha', 32)
            )

            state_dict_path = self.model_dir / f"ensemble_model_{i}.pt"
            state_dict = torch.load(state_dict_path, map_location=self.device)
            model.load_state_dict(state_dict)

            model.to(self.device)
            model.eval()

            weight = weights[i] if i < len(weights) else 1.0
            models.append({'model': model, 'weight': weight, 'name': model_name})

        print(f"Loaded {len(models)} models successfully.")
        return models

    def load_test_data(self, csv_path, img_dir):
        test_files = []
        with open(csv_path, 'r') as f:
            reader = csv.reader(f)
            for row in reader:
                if row:
                    img_name = row[0].strip()
                    img_path = Path(img_dir) / img_name
                    if img_path.exists():
                        test_files.append(str(img_path))
        print(f"Loaded {len(test_files)} test images.")
        return test_files

    def get_tta_transforms(self, img_size=224, n_aug=5):
        transforms = [
            T.Compose([
                ResizeOrPad(img_size), T.Resize((img_size, img_size)), T.ToTensor(),
                T.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
            ])
        ]
        for i in range(n_aug - 1):
            resize_delta = random.choice([-20, -10, 0, 10, 20])
            target_size = max(img_size, img_size + resize_delta)
            transforms.append(
                T.Compose([
                    ResizeOrPad(target_size + 20),
                    T.Resize((target_size + 10, target_size + 10)),
                    T.CenterCrop(img_size) if i % 2 == 0 else T.RandomCrop(img_size),
                    T.RandomApply([T.ColorJitter(brightness=random.uniform(0.1, 0.2), contrast=random.uniform(0.1, 0.2), saturation=random.uniform(0.05, 0.15), hue=random.uniform(0.02, 0.05))], p=0.8),
                    T.RandomChoice([T.GaussianBlur(3, sigma=(0.1, 0.5)), nn.Identity()]),
                    T.ToTensor(),
                    T.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
                ])
            )
        return transforms

    def predict_single_image_tta(self, model, img_path, n_aug=5):
        image = Image.open(img_path).convert('RGB')
        tta_transforms = self.get_tta_transforms(n_aug=n_aug)
        aug_probs = []
        with torch.no_grad():
            for transform in tta_transforms:
                img_tensor = transform(image).unsqueeze(0).to(self.device)
                primary_output, _ = model(img_tensor)
                probs = F.softmax(primary_output, dim=1)
                aug_probs.append(probs)

        weights = torch.tensor([1.5] + [1.0] * (n_aug - 1)).to(self.device)
        weights = weights / weights.sum()
        final_probs = torch.zeros_like(aug_probs[0])
        for i, probs in enumerate(aug_probs):
            final_probs += probs * weights[i]
        return final_probs.cpu().numpy().squeeze()

    def ensemble_predict_batch(self, models, test_files, use_tta=True):
        predictions = {}
        for img_path in tqdm(test_files, desc="Predicting"):
            img_name = Path(img_path).name
            all_model_probs, model_weights = [], []
            for model_info in models:
                model, weight = model_info['model'], model_info['weight']
                if use_tta:
                    probs = self.predict_single_image_tta(model, img_path, n_aug=3)
                else:
                    transform = T.Compose([
                        ResizeOrPad(224), T.Resize((224, 224)), T.ToTensor(),
                        T.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
                    ])
                    image = Image.open(img_path).convert('RGB')
                    img_tensor = transform(image).unsqueeze(0).to(self.device)
                    with torch.no_grad():
                        primary_output, _ = model(img_tensor)
                        probs = F.softmax(primary_output, dim=1).cpu().numpy().squeeze()
                all_model_probs.append(probs)
                model_weights.append(weight ** 2)

            model_weights = np.array(model_weights)
            model_weights /= model_weights.sum()
            final_probs = np.sum([p * w for p, w in zip(all_model_probs, model_weights)], axis=0)
            pred_class = np.argmax(final_probs)
            predictions[img_name] = int(pred_class)
        return predictions

    def predict_and_save(self, csv_path, img_dir, output_path, use_tta=True):
        models = self.load_ensemble_models()
        if not models: return
        test_files = self.load_test_data(csv_path, img_dir)
        predictions = self.ensemble_predict_batch(models, test_files, use_tta=use_tta)
        print(f"\nSaving results to {output_path}")
        with open(output_path, 'w') as f:
            json.dump(predictions, f, indent=2)
        print(f"Saved {len(predictions)} predictions")
        return predictions

In [ ]:
model_dir = "/kaggle/input/bestmodel"
csv_path = "/kaggle/input/ent-private-test/ENTRep_Track1_Private_Data/cls.csv"
img_dir = "/kaggle/input/ent-private-test/ENTRep_Private_Dataset_Update/imgs"
output_path = "best_predictions.json"

predictor = TestPredictor(model_dir)
predictor.predict_and_save(
    csv_path=csv_path,
    img_dir=img_dir,
    output_path=output_path,
    use_tta=True
)
print(f"\nResults saved to: {output_path}")